<a href="https://colab.research.google.com/github/Uriel1201/HelloSQL/blob/main/12_Hacker_scores/12_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 12. Hacker scores
Given the following two tables, write a query to return the hacker ID, name, and total score (the sum of maximum scores for each challenge completed) ordered by descending score, and by ascending hacker ID in the case of score tie. Do not display entries for hackers with a score of zero.

In [1]:
!wget https://raw.githubusercontent.com/Uriel1201/HelloSQL/main/function_df.py

--2024-05-10 17:52:34--  https://raw.githubusercontent.com/Uriel1201/HelloSQL/main/function_df.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1237 (1.2K) [text/plain]
Saving to: ‘function_df.py’

function_df.py      100%[===================>]   1.21K  --.-KB/s    in 0s      

2024-05-10 17:52:34 (42.5 MB/s) - ‘function_df.py’ saved [1237/1237]



In [2]:
import pandas as pd
import numpy  as np
import function_df as f_df

data1 = {'hacker_id' : [1,2,3,4],
         'name'      : ['John','Jane','Joe','Jim']
        }

data2 = {'sub_id'       : [101,102,103,104,105,106,107,108,109],
         'hacker_id'    : [1,1,2,2,2,3,3,3,4],
         'challenge_id' : [1,1,1,1,2,1,2,2,1],
         'score'        : [10,12,11,9,13,9,12,15,0]
        }

hackers     = pd.DataFrame(data1)
submissions = pd.DataFrame(data2)

In [3]:
hackers

,hacker_id,name
0,1,John
1,2,Jane
2,3,Joe
3,4,Jim


In [4]:
submissions

,sub_id,hacker_id,challenge_id,score
0,101,1,1,10
1,102,1,1,12
2,103,2,1,11
3,104,2,1,9
4,105,2,2,13
5,106,3,1,9
6,107,3,2,12
7,108,3,2,15
8,109,4,1,0


In [5]:
submissions = submissions.sort_values(by = ['hacker_id', 'challenge_id', 'score'])
submissions['score_rank'] = submissions.groupby(['hacker_id', 'challenge_id'])['score'].transform(f_df.desc_row_num)

In [6]:
submissions

,sub_id,hacker_id,challenge_id,score,score_rank
0,101,1,1,10,2
1,102,1,1,12,1
3,104,2,1,9,2
2,103,2,1,11,1
4,105,2,2,13,1
5,106,3,1,9,1
6,107,3,2,12,2
7,108,3,2,15,1
8,109,4,1,0,1


In [7]:
c = submissions['score_rank'] == 1
max_scores = submissions.loc[c, ['hacker_id', 'score']]
max_scores.set_index('hacker_id', inplace = True, drop = True)

In [8]:
total_score = max_scores.groupby(level=0).sum()
total_score = total_score[total_score['score'] > 0]

In [9]:
total_score

,score
hacker_id,
1,12
2,24
3,24


In [10]:
hackers.set_index('hacker_id', inplace = True, drop = True)

In [11]:
hackers

,name
hacker_id,
1,John
2,Jane
3,Joe
4,Jim


In [12]:
hacker_scores = pd.concat([hackers, total_score], axis = 1, join = 'inner')

In [13]:
hacker_scores.sort_values(by = 'score', ascending = False)

,name,score
hacker_id,,
2,Jane,24
3,Joe,24
1,John,12
